In [2]:
include("ABLBreIF/BLBreIF.jl")
include("SCNMF/SCNMF.jl")
using .BLBreIF
using .SCNMF

using SparseArrays

In [3]:
A = sprand(Float64, 200, 10, 0.8) * sprand(Float64, 10, 200, 0.8)
A = SCNMF.normalize!(A)
X, Y = SCNMF.randinit(A, 10, 0.2) # initialize the fatorization matrices

([0.0 0.0006386433180594667 … 0.0006891589050608355 0.0017848123848202655; 0.0021491759326979507 0.0001635932223740386 … 0.0018328271400225946 0.0016735343906521239; … ; 2.8989017580736996e-5 0.00016938102808157406 … 0.0014951931466227809 0.0; 0.000716299338081572 0.0 … 0.0007084604085140849 0.0], [0.0012880105116404317 0.0005252871350111065 … 0.0014456160561992395 0.0; 0.0 0.0003657766209200794 … 0.0017218029264479367 0.0002855191027392257; … ; 0.00025536836618893634 0.0014021612672113748 … 0.0009097974037265043 0.0018875147966200116; 0.0 0.0 … 0.000930014782628903 0.0])

In [8]:
using StatsBase
using Base
sqL2dist(A, X *Y)
zero_rate(X)

0.1875

In [9]:
ρ₀ = 0.8 # step size parameter
μ₀ = 0.02 # regularization cofficient
μ = 0.002 # another regularization coefficient
rtime = 30 # runtime

30

In [10]:
X₀ = copy(X);
Y₀ = copy(Y);
r₀ = SCNMF.solve!(SCNMF.BPG{Float64}(obj=:cons,
                constraint=0.2,
                runtime=rtime,
                ρ=ρ₀), A, X₀, Y₀)

X₁ = copy(X);
Y₁ = copy(Y);
r₁ = SCNMF.solve!(SCNMF.BBPG{Float64}(obj=:cons,
                constraint=0.2,
                runtime=rtime,
                ρ=ρ₀), A, X₁, Y₁)

X₂ = copy(X);
Y₂ = copy(Y);
r₂ = BLBreIF.solve!(BLBreIF.ALBreI{Float64}(runtime=rtime,
                verbose=true,
                ρ=ρ₀,
                μ₁=μ₀,
                μ₂=μ₀), A, X₂, Y₂)

X₃ = copy(X);
Y₃ = copy(Y);
r₃ = BLBreIF.solve!(BLBreIF.ABLBreI{Float64}(runtime=rtime,
                verbose=true,
                ρ=ρ₀,
                μ₁=μ₀,
                μ₂=μ₀), A, X₃, Y₃)
X₄ = copy(X);
Y₄ = copy(Y);
r₄ = BLBreIF.solve!(BLBreIF.ALBreI{Float64}(runtime=rtime,
                verbose=true,
                ρ=ρ₀,
                μ₁=μ,
                μ₂=μ), A, X₄, Y₄)
X₅ = copy(X);
Y₅ = copy(Y);
r₅ = BLBreIF.solve!(BLBreIF.ABLBreI{Float64}(runtime=rtime,
                verbose=true,
                ρ=ρ₀,
                μ₁=μ,
                μ₂=μ), A, X₅, Y₅)

Iter     Elapsed time     objv             objv.change      (X & Y).change
    0     0.000000e+00     4.985863e-01


    1     9.109998e-01     9.881874e-01     4.896010e-01     2.028195e-03
    2     1.150000e+00     9.515074e-01    -3.667994e-02     3.773880e-03


    3     1.207000e+00     8.372360e-01    -1.142715e-01     5.880425e-03
    4     1.247000e+00     6.290341e-01    -2.082019e-01     6.486617e-03
    5     1.300000e+00     4.121228e-01    -2.169113e-01     6.123041e-03


    6     1.349000e+00     2.528379e-01    -1.592849e-01     4.757310e-03
    7     1.395000e+00     1.544383e-01    -9.839958e-02     3.451613e-03
    8     1.433000e+00     9.868118e-02    -5.575711e-02     2.452365e-03


    9     1.480000e+00     6.853755e-02    -3.014363e-02     1.731955e-03


...

 2931     2.175900e+01     3.010404e-03    -2.434447e-07     4.851518e-05
 2932     2.176900e+01     3.010175e-03    -2.290633e-07     5.653947e-05
 2933     2.178200e+01     3.009996e-03    -1.788945e-07     3.538089e-05
 2934     2.179300e+01     3.009931e-03    -6.545248e-08     3.238112e-05
 2935     2.180500e+01     3.009655e-03    -2.753111e-07     3.964145e-05
 2936     2.182300e+01     3.009173e-03    -4.826060e-07     5.095241e-05
 2937     2.182800e+01     3.008765e-03    -4.079680e-07     6.169799e-05
 2938     2.183200e+01     3.008023e-03    -7.418677e-07     6.318239e-05
 2939     2.183700e+01     3.007266e-03    -7.563760e-07     1.625159e-04
 2940     2.184200e+01     3.006877e-03    -3.890458e-07     5.645113e-05
 2941     2.185100e+01     3.006636e-03    -2.409699e-07     4.836265e-05
 2942     2.185900e+01     3.006410e-03    -2.268240e-07     5.603421e-05


 2943     2.186700e+01     3.006232e-03    -1.780863e-07     3.530010e-05
 2944     2.187300e+01     3.006167e-03    -6.503216e-08     3.223617e-05
 2945     2.188000e+01     3.005892e-03    -2.745079e-07     3.946668e-05
 2946     2.189500e+01     3.005412e-03    -4.802785e-07     5.072564e-05
 2947     2.190300e+01     3.005007e-03    -4.051275e-07     6.141984e-05
 2948     2.191000e+01     3.004267e-03    -7.397285e-07     6.215171e-05
 2949     2.191700e+01     3.003523e-03    -7.437654e-07     1.572118e-04
 2950     2.192500e+01     3.003136e-03    -3.872077e-07     5.629982e-05
 2951     2.193100e+01     3.002897e-03    -2.385208e-07     4.820769e-05
 2952     2.193700e+01     3.002673e-03    -2.246645e-07     5.554910e-05
 2953     2.194300e+01     3.002495e-03    -1.773147e-07     3.521985e-05
 2954     2.194900e+01     3.002431e-03    -6.463772e-08     3.224803e-05
 2955     2.195500e+01     3.002157e-03    -2.738838e-07     3.929241e-05


Excessive output truncated after 524300 bytes.

 2956     2.196600e+01     3.001679e-03    -4.780090e-07     5.050032e-05
 2957     2.197100e+01     3.001276e-03    -4.023762e-07     6.114015e-05
 2958     2.197400e+01     3.000539e-03    -7.376292e-07     6.115819e-05
 2959     2.197700e+01     2.999807e-03    -7.318636e-07     1.521189e-04
 2960     2.198200e+01     2.999422e-03    -3.854363e-07     5.614980e-05
 2961     2.198800e+01     2.999185e-03    -2.360934e-07     4.805083e-05
 2962     2.199500e+01     2.998963e-03    -2.226050e-07     5.518554e-05


Main.BLBreIF.Result{Float64}([0.0 0.0005338235415089257 … 0.02544489418287112 0.01072104312680759; 0.03353084148379821 0.02216537705973365 … 0.007020725052024904 0.007748649718866664; … ; 0.004666059904529703 0.033512977851269755 … 0.014017269040665812 0.018974014232462692; 0.0 0.017099890942221637 … 0.000816522176854829 0.01975459349107838], [0.0260551202752638 0.009069891065740127 … 0.03194910442635021 0.059230803137073504; 0.03083681996114001 0.012866853010536018 … 0.029678374894939477 0.05059213502659332; … ; 0.014314101414324099 0.027487751028885492 … 0.0 0.005038083247682926; 0.0409699701418976 0.04214430699106183 … 0.011562502339301256 0.02553661490988205], 3982, false, [0.0 0.9971726613284133; 0.004000186920166016 0.9964034860176181; … ; 0.0 0.0; 0.0 0.0])

In [11]:
stop₀ = r₀.niters
pic₀ = r₀.objvalue
rt₀ = pic₀[1:stop₀, 1]
obj₀ = pic₀[1:stop₀, 2]

stop₁ = r₁.niters
pic₁ = r₁.objvalue
rt₁ = pic₁[1:stop₁, 1]
obj₁ = pic₁[1:stop₁, 2]

stop₂ = r₂.niters
pic₂ = r₂.objvalue
rt₂ = pic₂[1:stop₂, 1]
obj₂ = pic₂[1:stop₂, 2]

stop₃ = r₃.niters
pic₃ = r₃.objvalue
rt₃ = pic₃[1:stop₃, 1]
obj₃ = pic₃[1:stop₃, 2]

stop₄ = r₄.niters
pic₄ = r₄.objvalue
rt₄ = pic₄[1:stop₄, 1]
obj₄ = pic₄[1:stop₄, 2]

stop₅ = r₅.niters
pic₅ = r₅.objvalue
rt₅ = pic₅[1:stop₅, 1]
obj₅ = pic₅[1:stop₅, 2]

3982-element Vector{Float64}:
 0.9971726613284133
 0.9964034860176181
 0.9956271242929096
 0.9948781221092239
 0.9940890172583261
 0.9932701018830989
 0.9924819625201973
 0.991716445778097
 0.9909823858686244
 0.9901973138731177
 ⋮
 0.0026905172524962573
 0.0026904613934696474
 0.00269023628121722
 0.0026898470474391316
 0.0026895365357397748
 0.0026888511460055253
 0.002688481787227005
 0.002688250973641067
 0.002688122392031956

In [12]:
using CairoMakie
using Makie
using LaTeXStrings
using GLMakie
using Colors
using AlgebraOfGraphics
CairoMakie.activate!()
function speed()
        lines(rt₀, sqrt.(obj₀); color="#4063D8", label="BPG", linewidth=3, linestyle=:dashdot,
                figure=(; figure_padding=50, resolution=(1200, 800), font="sans",
                        backgroundcolor=:white, fontsize=32),
                axis=(; xlabel="Time(sec)", ylabel=L"\Vert A-XY\Vert_F",
                        #yscale=log10,
                        xgridstyle=:dash, ygridstyle=:dash))
        lines!(rt₁, sqrt.(obj₁); color="#389826", linewidth=3, linestyle=:dashdot,
                label="BBPG")
        lines!(rt₂, sqrt.(obj₂); color="#FFC633", linewidth=2, linestyle=:solid,
                label="ALBreI(μ=$μ₀)")
        lines!(rt₃, sqrt.(obj₃); color="#CB3C33", linewidth=2, linestyle=:solid,
                label="ABLBreI(μ=$μ₀)")
        lines!(rt₄, sqrt.(obj₄); color="#FFC633", linewidth=2, linestyle=:dot,
                label="ALBreI(μ=$μ)")
        lines!(rt₅, sqrt.(obj₅); color="#CB3C33", linewidth=2, linestyle=:dot,
                label="ABLBreI(μ=$μ)")
        limits!(0, rtime, 0.0, 1.0)
        axislegend("Algorithms"; merge=true)
        current_figure()
end

with_theme(speed, theme=theme_dark())
save("plot/synthesis_data.png", speed())

CairoMakie.Screen{IMAGE}


In [14]:
# 求矩阵X的零向量占比
function zero_rate(X)
        n = size(X, 1)
        m = size(X, 2)
        count = 0
        for i in 1:n
                for j in 1:m
                        if X[i, j] == 0
                                count += 1
                        end
                end
        end
        return count / (n * m)
end

zero_rate(X₂)

0.1875